In [3]:
import h5py
from druglab.storage import BaseStorage

class ExampleStorage(BaseStorage):
    @property
    def required_object_keys(self):
        return ['items']
    
    @property
    def save_dtypes(self):
        return {'items': h5py.string_dtype()}
    
    def __init__(self, items=None, **kwargs):
        objects = {'items': items or []}
        super().__init__(objects=objects, **kwargs)
    
    def get_save_ready_objects(self):
        return {'items': [str(item) for item in self._objects['items']]}
    
    def get_load_ready_objects(self, db, indices=None):
        if indices is None:
            data = db['items'][:]
        else:
            data = db['items'][indices]
        
        items = [item.decode() if isinstance(item, bytes) else str(item) 
                 for item in data]
        return {'items': items}

In [8]:
import tempfile
from pathlib import Path
import numpy as np

# Create storage with data
storage = ExampleStorage(['item1', 'item2', 'item3'])

# Add features
features = np.random.rand(3, 4)
storage.features.add_features('test_feat', features)

# Add metadata
storage.metadata['version'] = 1
storage.metadata['created_by'] = 'test'

with tempfile.NamedTemporaryFile(suffix='.h5', delete=False) as f:
    temp_path = f.name

# Save
storage.write(temp_path)

# Load
loaded_storage = ExampleStorage.load(temp_path)

# Verify objects
assert len(loaded_storage) == 3
assert loaded_storage._objects['items'] == ['item1', 
                                            'item2', 
                                            'item3']

# Verify features
loaded_features = loaded_storage.features.get_features('test_feat')
np.testing.assert_array_almost_equal(loaded_features, features)

Path(temp_path).unlink(missing_ok=True)

KeyError: 'test_feat'

In [7]:
storage.features.keys()

dict_keys(['test_feat'])